### Coding Discussion 5 _ Harshini 




Can we predict whether someone will vote or not?

In the discussion folder, you'll find the turnout.csv data, which was drawn from the 2012 National Election Survey. The data records the age, eduction level (of total years in school), income, race (caucasian or not), and past voting record (i.e. whether or not the respondent voted in the 2012 Presidential election). The sample is composed of 2000 individual respondents.

Please break the data up into a training (1600 entries, 80%) and test dataset (400 entries, 20%).

Build a Naive Bayesian Classifier from scratch that tries to predict whether a respondent will vote in a presidential election or not, pr(Vote==1). The classifier must be built from scratch. Do not use a third party ML or statistical package.

Run your algorithm and see how it predicts on the test data. Use the sklearn.metrics library to calculate the predictive accuracy and the area underneath the ROC curve.

Does your model perform better than chance (i.e. coin flip)?




- As asked, in this file, a Naive Bayesian classifier was built from scratch. It takes one input file, i.e., turnout. 

#### Loading packages

In [261]:
## Importing required packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt #for plotting
from random import shuffle #for shuffling the data before splitting it
from sklearn.naive_bayes import GaussianNB # Classifier 
import sklearn.metrics as m # sklearn.metrics to determine the classification 


#### Reading the data

In [262]:
##Loading data
turnout = pd.read_csv("C:/Users/VIOLIN/Desktop/Coding discussion 5/turnout.csv")
##Printing data
print(turnout)



        id  age  educate  income  vote  white
0        1   60     14.0  3.3458     1      1
1        2   51     10.0  1.8561     0      1
2        3   24     12.0  0.6304     0      1
3        4   38      8.0  3.4183     1      1
4        5   25     12.0  2.7852     1      1
...    ...  ...      ...     ...   ...    ...
1995  1996   26     16.0  3.3834     0      1
1996  1997   34     12.0  2.9170     1      1
1997  1998   51     16.0  7.8949     1      1
1998  1999   22     10.0  2.4811     0      1
1999  2000   59     10.0  0.5523     0      1

[2000 rows x 6 columns]


In [263]:
# Shuffling the data

turnout_r = turnout.sample(frac=1)
#Printing the shuffled data
print(turnout_r)


        id  age  educate   income  vote  white
998    999   27     14.0   8.7545     1      1
1779  1780   63      4.0   2.2092     0      1
650    651   24     12.0   1.4055     0      1
1987  1988   62      9.0   0.7294     0      0
1757  1758   49     14.0  11.2307     1      1
...    ...  ...      ...      ...   ...    ...
1322  1323   45     15.0   9.1907     1      1
909    910   42     12.0   8.7220     1      1
1432  1433   52      9.0   3.7042     1      1
330    331   62      6.0   2.9170     1      1
1772  1773   23     16.0   0.2364     1      1

[2000 rows x 6 columns]


#### Breaking the data into testing and training dataset



In [264]:

#Splitting the data into testing and training data
testing_dataset = turnout_r[0:400]
training_dataset = turnout_r[400:2000]
#Printing outputs to cross check 
print(testing_dataset)
print(training_dataset)

        id  age  educate   income  vote  white
998    999   27     14.0   8.7545     1      1
1779  1780   63      4.0   2.2092     0      1
650    651   24     12.0   1.4055     0      1
1987  1988   62      9.0   0.7294     0      0
1757  1758   49     14.0  11.2307     1      1
...    ...  ...      ...      ...   ...    ...
148    149   38      5.0   1.4498     1      1
1117  1118   39     14.0   7.5945     1      1
172    173   27     12.0   1.8429     1      1
25      26   39     16.0  10.0376     1      1
548    549   46      7.0   0.5523     1      0

[400 rows x 6 columns]
        id  age  educate  income  vote  white
1186  1187   28     12.0  2.9170     1      1
391    392   52     14.0  4.4876     1      1
1620  1621   36     12.0  4.0702     1      1
1763  1764   57     16.0  3.8606     0      1
1478  1479   66     14.0  0.7953     1      1
...    ...  ...      ...     ...   ...    ...
1322  1323   45     15.0  9.1907     1      1
909    910   42     12.0  8.7220     1      

 #### Naive Bayesian Classifier 

#### THEORY OF THE CLASSIFIER

#### - Bayes Theorem : P(A/B) = [P(B/A) * P(A)] / P(B)
#### - Applying the theorem to the current problem : P(y/X) = P(X/y) * P(y)/P(X)
##### Explaining the terms : X - feature vector i.e., X = {x1,x2,x3,x4,x5}x1-age ,x2-educate,x3-income,x4-white; y-vote ; 
##### P(y/X) - Posterior Probability;P(x1/y),P(x2/y),P(x3/y),P(x4/y),P(x5/y)- Class Conditional Probabilities; P(X);P(y) - Prior Probabilities;Prior - Frequency
#### As we are creating a Naive Bayesian Classifier, all the features(x1-x5) are considered to be mutually independent : P(X/y) = P(x1/y).P(x2/y).P(x3/y).P(x4/y).P(x5/y)

#### Calculating Class Conditional Probability : P(xi/y) =  (1/(2*pi*sigma_y^2))*exp(-(xi-mu_y)^2/(2*sigma_y^2);sigma - variance; mu - mean value


In [265]:

#creating a class NaiveBayes with necessary NB classifier functions loaded
class NaiveBayes:
    """
    The defined class NaiveBayes contains four functions defined,i.e., fit to calculate mean,variance and prior, predict to 
    calculate overall required prediction value ,_predict to calculate prediction value with one feature variable and 
    _pdf to help predict function
    
    """
    
    def fit(self,X,y): 
        
        n_rows,n_columns= X.shape #Since X is an array with rows and columns
        self._classes = np.unique(y) # to find the unique elements of the array
        n_classes =len(self._classes)
        #init mean, variance and prior
        self._mean = np.zeros((n_classes,n_columns),dtype =np.float64)
        self._var = np.zeros((n_classes,n_columns),dtype =np.float64)
        self._prior =np.zeros((n_classes),dtype =np.float64)
        
        for c in self._classes: #For class in self._classes
            X_c = X[c==y]
            self._mean[c,:] = X_c.mean(axis=0) # to calculate mean for each class
            self._var[c,:] = X_c.var(axis=0) # to calculate variance for each class
            self._prior[c] = X_c.shape[0]/float(n_rows) #frequency
            
    def predict(self,X):
        
        y_predict = [self._predict(x) for x in X] 
        return y_predict  # return output    
    
    def _predict(self,x):
       
        posterior = [] # Create an empty list 
        for idx, c in enumerate(self._classes):#idx -> index 
            prior = np.log(self._prior[idx]) #prior probability as log(for%)
            # calculating class cond prob using a _pdf function,defined below
            class_cond_prob = np.sum(np.log(self._pdf(idx,x))) 
            posterior1 =  class_cond_prob + prior #Summing class conditional prob's and prior prob
            posterior.append(posterior1) #Final Posterior Probability
            return self._classes[np.argmax(posterior)] #index with highest prob
        

            
    def _pdf(self, class_index, x):
        mean = self._mean[class_index]#calculating mu i.e., mean
        var = self._var[class_index] #calculating sigma,i.e., variance
        numerator = np.exp(- (x-mean)**2 / (2 * var))#numerator for class conditional prob
        denominator = np.sqrt(2* np.pi * var)#denominator for class conditional prob
        return numerator / denominator #output for calculating class conditional prob
            

#### Accuracy of the created NB Classifier

In [266]:
def accuracy(y_true, y_predict): 
    """
    The defined accuracy function helps us find out the accuracy of the created NB Classifier
    """
    accuracy = np.sum(y_true == y_predict) / len(y_true) # defining accuracy
    return accuracy #return output


In [270]:
nb = NaiveBayes() ##Calling the created class

y_train = training_dataset["vote"] #creating y(vote) dataset for training dataset
X_train = training_dataset.drop(columns=["vote"])#creating X feature vector dataset for training dataset
X_train= X_train.reset_index().values#to reset the index of dataset
y_test = testing_dataset["vote"]#creating y(vote) dataset for testing dataset
X_test = testing_dataset.drop(columns=["vote"])#creating X feature vector  dataset for  testing dataset
X_test= X_test.reset_index().values#to reset the index of dataset
nb.fit(X_train,y_train) #Calling the classifier for training data
prediction = nb.predict(X_test) #Prediction for testing data
prediction1 = nb.predict(X_train)#Prediction for trainining data

print(accuracy(y_test, prediction)) #Output #accuracy is too low for the testing data
print(accuracy(y_train, prediction1))#Output #accuracy is too low for the trainig data

0.2625
0.251875


#### Comparing the accuracy with Sklearn

In [268]:
#Accuracy calculation with Sklearn metrics
# Instantiate the model class 
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Prediction value
y_pred = gnb.predict(X_test)

# Accuracy with Sklearn
m.accuracy_score(y_test,y_pred) 

0.735

#### ROC Curve 

In [269]:

#Area under the ROC curve
m.roc_auc_score(y_test,y_pred) 


0.5811138014527846

#### Discussion

- AUC of the classifier is 0.598 while coin flip has 0.5 chance. So, the model performs 18% better than the coin flip.  
- Accuracy for the defined classifier is very low .Improving the method of sampling might address the issue. 
    
